# Notebook for ExpertsRS
This is a demo notebook for running our ExpertsRS prototype system


## Set up the API
Set up your LLM api and URL, user can include api for different models in the [config list](llm_config_list.json)

In [ ]:
import autogen

import sys
import os

# Add project root to sys.path
notebook_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(notebook_dir, ".."))
if project_root not in sys.path:
    sys.path.append(project_root)

from prompts import scientist_prompt, engineer_prompt, manager_prompt, user_proxy_prompt, executor_prompt

config_list = autogen.config_list_from_json("llm_config_list.json") # set the api key in the config list
print("LLM models: ", [config_list[i]["model"] for i in range(len(config_list))]) # print the list of LLM models

In [ ]:
# Build filters to select different LLM models based on tags
llm_filters = {
    "gpt-4": {"tags": ["gpt-4"]},
    "deepseek-v3": {"tags": ["deepseek-v3"]},
    "deepseek-r1": {"tags": ["deepseek-r1"]},
}

# Filter config_list for each model
gpt4_config_list = autogen.filter_config(config_list, llm_filters["gpt-4"])
deepseekv3_config_list = autogen.filter_config(config_list, llm_filters["deepseek-v3"])
deepseekr1_config_list = autogen.filter_config(config_list, llm_filters["deepseek-r1"])

# Ensure only one config matched for deepseek-v3
assert len(deepseekv3_config_list) == 1, "deepseek-v3 config_list must contain exactly 1 config."

# Build llm_config dict for each model
# gpt-4
gpt4_config = {
    "cache_seed": 1,
    "temperature": 0,
    "timeout": 180,
    "config_list": gpt4_config_list,
}

# deepseek-v3
deepseekv3_config = {
    "cache_seed": 100,
    "temperature": 0,
    "timeout": 180,
    "config_list": deepseekv3_config_list,
}

# deepseek-r1
deepseekr1_config = {
    "cache_seed": 1,
    "temperature": 0,
    "timeout": 180,
    "config_list": deepseekr1_config_list,
}

# Double-check the model
print("[INFO] LLM models in config_list:")
print(" - GPT-4:", [cfg["model"] for cfg in gpt4_config_list])
print(" - DeepSeek-V3:", [cfg["model"] for cfg in deepseekv3_config_list])
print(" - DeepSeek-R1:", [cfg["model"] for cfg in deepseekr1_config_list])


## Construct Agents

In [20]:
# The User proxy
user_proxy = autogen.UserProxyAgent(
    name="User",
    system_message= user_proxy_prompt,
    code_execution_config=False,
    human_input_mode="ALWAYS",
)

# The Manager
manager = autogen.AssistantAgent(
    name="Manager",
    system_message= manager_prompt,
    llm_config=deepseekv3_config
)

# The Scientist
scientist = autogen.AssistantAgent(
    name="Scientist",
    system_message= scientist_prompt,
    llm_config=deepseekv3_config
)

# The Engineer
engineer = autogen.AssistantAgent(
    name="Engineer",
    system_message=engineer_prompt,
    llm_config=deepseekv3_config,
    code_execution_config=False,  # Turn off code execution for this agent.

)

# The Executor
executor = autogen.UserProxyAgent(
    name="Executor",
    system_message=executor_prompt,
    human_input_mode="NEVER",  # auto response mode
     code_execution_config={
        "executor": "commandline-local",  # use local environment
        "last_n_messages": 3,
    }
)


## Customize agent selction function
Inspired by [StateFlow](https://arxiv.org/abs/2403.11322), a LLM-based task-solving paradigm that conceptualizes complex task-solving processes as state machines.

ExpertsRS operates through **four states** : 
- 1. Clarify request: The Manager engages with the User to transform vague inputs into a structured request
- 2. Define problem: The Scientist formulates a detailed statement outlining objectives, data needs, and methods
- 3. Solve problem: The engineer agent breaks the statement into subtasks, generates executable code, and submits it to the Executor. If errors arise, messages are routed to the Engineer for iterative code refinement until successful execution.
- 4. Generate report: The Manager compiles a user-friendly report incorporating the analytical process, maps, indicators, and other explainations.

Check section 3.1 and 4.1 in the paper for details.

In [21]:
# Customized agent selection function
def speaker_selection(last_speaker, groupchat):
    messages = groupchat.messages
    
    if len(messages) <= 1:  
        # Initial state: user starts conversation with Manager
        return manager
    
    if last_speaker == manager:
        # Manager communicates directly with the user
        return user_proxy
    
    if last_speaker == user_proxy:
        # After Manager's communication, if user approves the report
        if "Approve" in messages[-1]["content"]:
            return scientist  # Pass the report to Scientist upon user approval
        elif "End" in messages[-1]["content"]:
            return None  # End the chat if user requests
        return manager  # Continue conversation with Manager if user hasn't approved
    
    # State 2: Transition from Scientist to Engineer for a well-defined research question
    if last_speaker == scientist:
        # Scientist hands over the task to Engineer
        return engineer
    
    if last_speaker == engineer:
        # Engineer generates code and passes it to Executor for execution
        return executor
    
    if last_speaker == executor:
        # Executor executes the code and checks for errors
        if any(keyword in messages[-1]["content"] for keyword in ["error", "failed", "exception"]):
            return engineer  # Return to Engineer for modifications if there are errors
        elif messages[-1]["content"] == "":
            return engineer  # Return to Engineer if output is empty
        else:
            return manager  # Return results to Manager if execution is successful

    return None  # End the conversation if none of the above conditions are met


In [22]:
# Group chat setting
groupchat = autogen.GroupChat(
    agents=[user_proxy, manager, scientist, engineer, executor], 
    messages=[], 
    max_round=30,
     speaker_selection_method=speaker_selection
)


# chat administrator (group chat manager)
chat_admin = autogen.GroupChatManager(
    name="Chat_Admin",
    llm_config=deepseekv3_config,
    groupchat = groupchat
)

## Start chatting
Input your request and run this cell. 

Chat with the Manager to express your need sufficiently, if satisfied, type 'Approve' to enter the following states.

Type 'End' or 'exit' to end the chat.


In [ ]:
# initiating chat
users_request = 'I am an official in the urban greening department. I want a map of greenspace to support my policy-making.' #TODO: enter user's request here

user_proxy.initiate_chat(
    chat_admin,
    message= users_request,
)